In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/Spotify-2000.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Index                   1994 non-null   int64 
 1   Title                   1994 non-null   object
 2   Artist                  1994 non-null   object
 3   Top Genre               1994 non-null   object
 4   Year                    1994 non-null   int64 
 5   Beats Per Minute (BPM)  1994 non-null   int64 
 6   Energy                  1994 non-null   int64 
 7   Danceability            1994 non-null   int64 
 8   Loudness (dB)           1994 non-null   int64 
 9   Liveness                1994 non-null   int64 
 10  Valence                 1994 non-null   int64 
 11  Length (Duration)       1994 non-null   object
 12  Acousticness            1994 non-null   int64 
 13  Speechiness             1994 non-null   int64 
 14  Popularity              1994 non-null   int64 
dtypes: i

In [13]:
categories = {}
for genre in data['Top Genre']:
    words = genre.split(' ')
    for word in words:
        if word in categories:
            categories[word] += 1
        else:
            categories[word] = 1
{k: v for k, v in sorted(categories.items(), key=lambda item: item[1], reverse=True)}

{'rock': 856,
 'album': 413,
 'pop': 344,
 'dutch': 266,
 'alternative': 187,
 'dance': 151,
 'adult': 123,
 'standards': 123,
 'classic': 95,
 'metal': 93,
 'indie': 81,
 'british': 74,
 'glam': 66,
 'modern': 56,
 'art': 54,
 'cabaret': 51,
 'soul': 45,
 'wave': 40,
 'permanent': 38,
 'irish': 36,
 'invasion': 36,
 'hip': 30,
 'hop': 29,
 'mellow': 28,
 'folk': 27,
 'europop': 27,
 'uk': 23,
 'blues': 23,
 'neo': 19,
 'disco': 18,
 'big': 18,
 'funk': 18,
 'country': 17,
 'australian': 17,
 'canadian': 14,
 'americana': 13,
 'celtic': 13,
 'belgian': 12,
 'reggae': 12,
 'german': 11,
 'britpop': 11,
 'gold': 11,
 'beat': 11,
 'carnaval': 10,
 'limburg': 10,
 'arkansas': 9,
 'chanson': 9,
 'electro': 8,
 'detroit': 7,
 'room': 7,
 'chamber': 7,
 'boy': 6,
 'band': 6,
 'east': 6,
 'coast': 6,
 'brill': 6,
 'building': 6,
 'baroque': 6,
 'prog': 5,
 'g': 5,
 'garage': 4,
 'latin': 4,
 'downtempo': 4,
 'fusion': 4,
 'acoustic': 4,
 'singer-songwriter': 4,
 'folk-pop': 4,
 'metropopolis':